In [28]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from utilities import *
import matplotlib.pyplot as plt
import os
from PIL import Image


# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Generate random noise as input to the generator
def generate_random_noise(batch_size, latent_dim):
    return np.random.normal(0, 1, size=(batch_size, latent_dim))

# Generator model (updated for 64x64 images)
def build_generator(latent_dim):
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_dim=latent_dim, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(64 * 64, activation='sigmoid'))  # Output size for 64x64 images
    model.add(layers.Reshape((64, 64, 1)))  # Reshape to image size (64x64x1)
    return model

# Discriminator model (updated for 64x64 images)
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Flatten(input_shape=(64, 64, 1)))  # Input size for 64x64 images
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# GAN model combining generator and discriminator
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = tf.keras.Sequential([generator, discriminator])
    return model
    
# Loss functions for generator and discriminator
cross_entropy = tf.keras.losses.BinaryCrossentropy()


def generator_loss(y_true, y_pred):
    return cross_entropy(tf.ones_like(y_pred), y_pred)
    
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def save_generated_images(generator, epoch, latent_dim, num_samples=10):
    noise = generate_random_noise(num_samples, latent_dim)
    generated_images = generator.predict(noise)
    generated_images = generated_images * 255.0  # Scale back to pixel values (0-255)
    generated_images = generated_images.astype('uint8')

    # Plot and save generated images
    plt.figure(figsize=(10, 1))
    for i in range(num_samples):
        plt.subplot(1, num_samples, i+1)
        plt.imshow(generated_images[i].squeeze(), cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f'generated_images_epoch_{epoch}.png')
    plt.close()

    
# Training the GAN
def train_gan(generator, discriminator, gan, real_data, epochs=10000, batch_size=64, latent_dim=100):
    for epoch in range(epochs):
        # Train discriminator
        real_samples = real_data[np.random.randint(0, real_data.shape[0], size=batch_size)]
        noise = generate_random_noise(batch_size, latent_dim)
        generated_samples = generator.predict(noise)
        
        real_labels = np.ones((batch_size, 1))
        fake_labels = np.zeros((batch_size, 1))

        d_loss_real = discriminator.train_on_batch(real_samples, real_labels)
        d_loss_fake = discriminator.train_on_batch(generated_samples, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train generator
        noise = generate_random_noise(batch_size, latent_dim)
        valid_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, valid_labels)

        if epoch % 1000 == 0:
            generated_images = generator.predict(noise)
            generated_images = generated_images * 255.0  # Scale back to pixel values (0-255)
            generated_images = generated_images.astype('uint8')
            plt.figure(figsize=(10, 1))
            num_samples=10
            for i in range(num_samples):
                plt.subplot(1, num_samples, i+1)
                plt.imshow(generated_images[i].squeeze(), cmap='gray')
                plt.axis('off')
            plt.tight_layout()
            plt.savefig(f'generated_images_epoch_{epoch}.png')
            plt.close()
            print(f"Epoch: {epoch}, Generator Loss: {g_loss}, Discriminator Loss: {d_loss}")

# Main code
# Load or create your dataset of real images (e.g., from MNIST)
# For simplicity, we'll use the MNIST dataset
(x_train_mnist, _), (_, _) = tf.keras.datasets.mnist.load_data()
x_train_mnist = x_train_mnist.reshape(x_train_mnist.shape[0], 28, 28, 1).astype('float32') / 255.0
# Resize images to 64x64
x_train_mnist = tf.image.resize(x_train_mnist, (64, 64))





# Load and preprocess your custom dataset from HDF5
with h5py.File('datasets/trainset.hdf5', 'r') as h5_file:
    custom_data = h5_file['X_train'][:]  # Replace 'your_dataset_name' with the dataset name in the HDF5 file
    custom_data = custom_data.astype('float32') / 255.0  # Normalize pixel values to [0, 1]
    custom_data = np.expand_dims(custom_data, axis=-1)  # Add channel dimension (grayscale)

# Combine MNIST with custom dataset
combined_data = np.concatenate((x_train_mnist, custom_data), axis=0)

# Shuffle the combined dataset
np.random.shuffle(combined_data)


# Set up GAN components
latent_dim = 100
generator = build_generator(latent_dim)
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.5))
gan.compile(loss=generator_loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.5))



# Train the GAN
train_gan(generator, discriminator, gan, combined_data)


2/2 [==============================] - 0s 2ms/step
Epoch: 0, Generator Loss: 1.0424216985702515, Discriminator Loss: 0.5637194514274597
2/2 [==============================] - 0s 2ms/step
Epoch: 1000, Generator Loss: 9.704291733214632e-05, Discriminator Loss: 4.956870913505554
2/2 [==============================] - 0s 2ms/step
Epoch: 2000, Generator Loss: 9.704291733214632e-05, Discriminator Loss: 4.960756897926331
2/2 [==============================] - 0s 3ms/step
Epoch: 3000, Generator Loss: 9.704291733214632e-05, Discriminator Loss: 4.957410663366318
2/2 [==============================] - 0s 2ms/step
Epoch: 4000, Generator Loss: 9.704291733214632e-05, Discriminator Loss: 4.969445198774338
2/2 [==============================] - 0s 2ms/step
Epoch: 5000, Generator Loss: 9.704291733214632e-05, Discriminator Loss: 4.963884651660919
2/2 [==============================] - 0s 3ms/step


KeyboardInterrupt: 